In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/range_bars_BTCUSDT_simple_strategy_indicators-alt.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)  
df.reset_index(drop=True, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(by='timestamp')
df['ordinal'] = df.index + 1
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df

In [ ]:
def get_start_end_idx_for_period(df: pd.DataFrame, resample_arg: str = 'M') -> str:
        # resample the dataframe into monthly periods
        groups = df.resample(resample_arg)
        # create a list of tuples containing the start and end indices for each period
        start_end_indices = []
        # iterate over each group (period) in the resampled DataFrame
        for period_start, group_df in groups:
            # get the start and end indices of the rows within the current period
            period_start = group_df.index[0]
            period_end = group_df.index[-1]
            # add the start and end indices to the list
            start_end_indices.append((period_start, period_end))
        return start_end_indices

In [ ]:
start_end_indices = get_start_end_idx_for_period(df)
start_end_indices

In [ ]:
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.1
    known_false_signals = True

    def init(self):
       self.false_buys = 0
       self.false_sells = 0

    def next(self):
        current_close = self.data.Close[-1]
        index = self.data.index[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]

        known_false_pass = True
        if self.known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size , tp=current_close - potential_profit)


     


In [ ]:
from backtesting.lib import TrailingStrategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarTrailingStrategyPreCalc(TrailingStrategy):

    per_trade_risk = 0.1
    known_false_signals = True

    def init(self):
       super().init()
       self.false_buys = 0
       self.false_sells = 0
       self.set_atr_periods(3)
       self.set_trailing_sl(1)

    def next(self):
        super().next()
        current_close = self.data.Close[-1]
        index = self.data.index[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]

        known_false_pass = True
        if self.known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size, tp=current_close - potential_profit)
     


In [ ]:
def backtest_period(strat, df, start, end, plot = True, leverage = 20):
    from backtesting import Backtest
    # from time import sleep 
    df_sample = df[start:end].copy()    
    bt = Backtest(df_sample, strat, cash=100_000, margin=1/leverage, commission=(0.03 / 100), exclusive_orders=False)
    stats = bt.run()
    # sleep(30)
    false_buys = stats._strategy.false_buys
    false_sells = stats._strategy.false_sells
    trades = stats['# Trades']
    return_percentage = stats['Return [%]']
    win_rate_percentage = stats['Win Rate [%]']
    profit_factor = stats['Profit Factor']
    profit = stats['Equity Final [$]'] - 100000
    max_drawdown_percentage = stats['Max. Drawdown [%]']
    average_drawdown_percentage = stats['Avg. Drawdown [%]']
    percentage_false_trades = round(((false_buys + false_sells) / int(trades)) * 100, 2)
    return {
        "profit": profit,
        "start": str(start),
        "end": str(end),
        "return_percentage": return_percentage,
        "win_rate_percentage": win_rate_percentage,
        "profit_factor": profit_factor,
        "max_drawdown_percentage": max_drawdown_percentage,
        "average_drawdown_percentage": average_drawdown_percentage,
        "trades": trades,
        "false_buys": false_buys,
        "false_sells": false_sells,
        "percentage_false_trades": percentage_false_trades
    }




errors don't seem to occur on a particular period, as demonstrated by the following code:

In [ ]:
# def test_period(idx: int):
#     import json
#     start, end = start_end_indices[idx]
#     print(f'start: {str(start)}, end: {str(end)}')
#     result = backtest_period(RangeBarTrailingStrategyPreCalc, df, start, end)
#     print(json.dumps(result, indent=4))

# test_period(1)    

In [ ]:
def test_periods(start_end_indices):
    # import json
    bulk_stats = []
    bulk_stats_trail = []
    errors = []
    for start, end in start_end_indices:
        print(f'start: {str(start)}, end: {str(end)}')
        try:
            result = backtest_period(RangeBarStrategyPreCalc, df, start, end, False)
            # print(json.dumps(result, indent=4))
            bulk_stats.append(result)
        except Exception as e:
            errors.append(f'strat: RangeBarStrategyPreCalc,  start: {str(start)}, end: {str(end)}. error: {str(e)}')
        
        # try: False
        #     result2 = trail_backtest_period(df, start, end, 20)
        #     # print(json.dumps(result, indent=4))
        #     bulk_stats_trail.append(result2)
        # except Exception as e:
        #     errors.append(f'strat: RangeBarTrailingStrategyPreCalc start: {str(start)}, end: {str(end)}. error: {str(e)}')    
        

    print(f'errors: {len(errors)}')
    print(f'errors: {errors}')
    df_stats = pd.DataFrame(bulk_stats)
    df_stats_trail = pd.DataFrame(bulk_stats_trail)
    return df_stats, df_stats_trail

df_stats, _ = test_periods(start_end_indices)

In [ ]:
tot_profit = df_stats['profit'].sum()
tot_profit_percentage = round((tot_profit / 100000) * 100, 2)
print(f'tot_profit_percentage: {tot_profit_percentage}')


* v1: tot_profit_percentage: 903.04
* v2: tot_profit_percentage: very bad
* v3: tot_profit_percentage: very bad

In [ ]:

# tot_profit_trial = df_stats_trail['profit'].sum()
# tot_profit_percentage_trail = round((tot_profit_trial / 100000) * 100, 2)
# print(f'tot_profit_percentage_trail: {tot_profit_percentage_trail}')

margin 1/20 tot pp: 903.04 (false)  
margin 1/20 tot pp: 10274.68 (no false)  

how to stop losses from large drawdown ?

In [ ]:
sorted_df_stats = df_stats.sort_values(by='profit_factor')
# sorted_df_stats.to_csv('data/std_false.csv')
sorted_df_stats 

In [ ]:
# sorted_df_stats = df_stats_trail.sort_values(by='profit_factor')
# sorted_df_stats 

# Biggest Loosers

* 2022-10-01 01:29:00	2022-10-31 23:08:00	-37.523810  	
* 2022-04-02 00:49:00	2022-04-30 03:48:00	-37.687021  	

In [ ]:
start1 = pd.to_datetime("2022-10-01 01:29:00")
end1 = pd.to_datetime("2022-10-31 23:08:00")
start2 = pd.to_datetime("2022-04-02 00:49:00")
end2 = pd.to_datetime("2022-04-30 03:48:00")
df_stats_2, _ = test_periods([[start1,end1], [start2, end2]])
sorted_df_stats_2 = df_stats_2.sort_values(by='profit_factor')
sorted_df_stats_2 

# Biggest Winners

* 2023-02-01 14:49:00	2023-02-28 08:28:00	189.561284
* 2023-01-01 00:02:00	2023-01-31 13:48:00	353.773942

In [ ]:
startb1 = pd.to_datetime("2023-02-01 14:49:00")
endb1 = pd.to_datetime("2023-02-28 08:28:00")
startb2 = pd.to_datetime("2023-01-01 00:02:00")
endb2 = pd.to_datetime("2023-01-31 13:48:00")

df_stats_3, _ = test_periods([[startb1, endb1], [startb2, endb2]])
sorted_df_stats_3 = df_stats_2.sort_values(by='profit_factor')
sorted_df_stats_3 